In [1]:
import numpy as np
import pandas as pd
#import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import operator
import warnings
warnings.filterwarnings("ignore")
from pandas.plotting import scatter_matrix
import re

In [2]:
df_IBTrACS = pd.read_csv('E:\WFP_Omdena_Project\TCE-DAT and IBTtracs merge\IBTrACS-ALL-list-v04r00-lines-dbf_Copy.csv')

In [3]:
df_IBTrACS_g1949 =df_IBTrACS[df_IBTrACS['year']>= 1949]
df_IBTrACS_g1949.reset_index(drop=True,inplace=True)

In [4]:
def get_perc_values(df):
  return sorted({k:round(v/df.shape[0],3) for k,v in dict(df.notnull().sum()).items()}.items(), key=operator.itemgetter(1), reverse=True)

sorted_perc_values = get_perc_values(df_IBTrACS)
sorted_perc_values

[('SID', 1.0),
 ('SEASON', 1.0),
 ('NUMBER', 1.0),
 ('NAME', 1.0),
 ('ISO_TIME', 1.0),
 ('NATURE', 1.0),
 ('LAT', 1.0),
 ('LON', 1.0),
 ('TRACK_TYPE', 1.0),
 ('DIST2LAND', 1.0),
 ('LANDFALL', 1.0),
 ('IFLAG', 1.0),
 ('NEW_CI', 1.0),
 ('REU_TNUM', 1.0),
 ('REU_CI', 1.0),
 ('BOM_TNUM', 1.0),
 ('BOM_CI', 1.0),
 ('STORM_SPD', 1.0),
 ('STORM_DR', 1.0),
 ('year', 1.0),
 ('month', 1.0),
 ('day', 1.0),
 ('hour', 1.0),
 ('min', 1.0),
 ('USA_SSHS', 0.992),
 ('SUBBASIN', 0.869),
 ('BASIN', 0.827),
 ('USA_ATCFID', 0.629),
 ('USA_LAT', 0.625),
 ('USA_LON', 0.625),
 ('USA_WIND', 0.532),
 ('TD6_STAGE', 0.448),
 ('TD6_LAT', 0.439),
 ('TD6_LON', 0.439),
 ('USA_STATUS', 0.322),
 ('USA_AGENCY', 0.307),
 ('WMO_AGENCY', 0.295),
 ('DS8_STAGE', 0.277),
 ('DS8_LAT', 0.273),
 ('DS8_LON', 0.273),
 ('DS8_WIND', 0.236),
 ('WMO_PRES', 0.212),
 ('WMO_WIND', 0.209),
 ('USA_PRES', 0.198),
 ('CMA_CAT', 0.195),
 ('CMA_LAT', 0.193),
 ('CMA_LON', 0.193),
 ('CMA_PRES', 0.193),
 ('TOK_LAT', 0.18),
 ('TOK_LON', 0.18),
 ('TO

In [5]:
df_IBTrACS_f = df_IBTrACS[df_IBTrACS['SEASON']>= 1950]
df_IBTrACS_f.reset_index(drop=True,inplace=True)

In [7]:
# df_IBTrACS.set_index(['SEASON','NAME','NUMBER'])

In [8]:
df_IBTrACS_land = df_IBTrACS[df_IBTrACS['DIST2LAND']==0]

In [11]:
import reverse_geocoder as rg

In [29]:
def add_country(lat, lon):
  coordinates = [(lat, lon)]
  results = rg.search(coordinates) # default mode = 2
  country = results[0]['cc']
  return country

df_IBTrACS_land_country = df_IBTrACS_land.copy()
df_IBTrACS_land_country['country'] = df_IBTrACS_land_country.apply(lambda x: add_country(x['LAT'], x['LON']), axis=1)

In [13]:
ibtracs = pd.read_csv('E:/WFP_Omdena_Project/TCE-DAT and IBTtracs merge/df_IBTrACS_ISO.csv')

In [16]:
import pandas as pd
import numpy as np
tcedf = pd.read_csv('E:/WFP_Omdena_Project/TCE-DAT and IBTtracs merge/TCE-DAT_2015-exposure_1950-2015.csv',skiprows = 6)

In [18]:
df_IBTrACS = df_IBTrACS[df_IBTrACS['year'] >= 1980]
tup_zipped = list(set(tuple(zip(df_IBTrACS.SID, df_IBTrACS.NAME))))

## Create a dictionary with ibtracs SID & storm name
SID_Namedict = {x:y for x, y in tup_zipped }

In [19]:
df_IBTrACS_copy = tcedf.copy()

## Create column "name" using ibtracID
df_IBTrACS_copy['name'] = df_IBTrACS_copy['IBTrACS_ID'].apply(lambda x: SID_Namedict.get(str(x)))

## Drop rows without event name
# df_IBTrACS_copy = df_IBTrACS_copy[df_IBTrACS_copy['name'] != "NOT_NAMED"]

# Drop rows with no names on both name & tc_name
# df_IBTrACS_copy = df_IBTrACS_copy[(df_IBTrACS_copy['TC_name'] != "UNNAMED") & (df_IBTrACS_copy['name'].notna())]

In [22]:
tcedf_copy = tcedf.copy()

## Create column "name" using ibtracID
tcedf_copy['name'] = tcedf_copy['IBTrACS_ID'].apply(lambda x: SID_Namedict.get(str(x)))

## Drop rows without event name
tcedf_copy = tcedf_copy[tcedf_copy['name'] != "NOT_NAMED"]

# Drop rows with no names on both name & tc_name
tcedf_copy = tcedf_copy[(tcedf_copy['TC_name'] != "UNNAMED") & (tcedf_copy['name'].notna())]

In [23]:
tcedf_copy.to_csv("tcedf_merge.csv")

In [24]:
ibtracs = ibtracs[ibtracs['SEASON']>= 1980]

In [25]:
merged_doc = pd.merge(tcedf_copy, ibtracs, how='outer',left_on=['year', 'IBTrACS_ID', 'ISO3'],right_on=['SEASON','SID','ISO'])

In [28]:
merged_doc.to_csv("merged_doc.csv")

In [35]:
# ibtracs_filtered = ibtracs_filtered.columns.str.contains('^test', case=False)

# ibtracs_filtered = ibtracs.loc[:,ibtracs.columns.str.contains('WIND', case=False)] 
# ibtracs_filtered = ibtracs_filtered[ibtracs_filtered.columns.drop(list(ibtracs_filtered.filter(regex='PRES')))]

In [40]:
# ibtracs_filtered = ibtracs[['SID','SEASON','NUMBER','BASIN','SUBBASIN','NAME','ISO_TIME','NATURE','LAT','LON','WMO_WIND','WMO_PRES','WMO_AGENCY','TRACK_TYPE','DIST2LAND','LANDFALL','IFLAG','STORM_SPD','STORM_DR','year','month','day','hour','min','country','ISO'
# ]]

In [41]:
merged_doc_filtered = merged_doc.groupby(['SID','ISO']).max().reset_index()

In [42]:
merged_doc_filtered.to_csv("merged_doc_filtered.csv")